### Data Cleaning for KAP Surveys - Bringing in Demographic Data

##### Bringing Demographic data into account for combined KAP Survey #1 and #2 results and creating a social desirability index
##### Bringing SGD data and created social desirability index together 

###### 15/07/2021 at 04:05 CEST

In [1]:
# import necessary libraries
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-pastel')

In [2]:
# change working directory to a specified directory
os.chdir('../')
print("Directory Changes")

# Get current working directory
cwd = os.getcwd()
print("Current working directory is:", cwd)

Directory Changes
Current working directory is: C:\Users\Carol\Documents\EPA_2020_2021\Thesis\Analysis\KAP_Analysis


In [36]:
# read in csv data
kap12 = pd.read_csv("data/interim/kap12_practice_clean.csv")
sd = pd.read_csv("data/raw/SocialDesirabilitySurvey_Testing_July142021.csv")
sgd= pd.read_csv("data/interim/sgd_practice_clean.csv")

## Prepare Demographic Data

In [37]:
# gain insight into sd
sd.shape

(11, 35)

In [38]:
# gain insight into kap1
sd.head(5)

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Random ID
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,"I can remember ""playing sick"" to get out of so...",There have been occasions when I have taken ad...,I sometimes try to get even rather than forgiv...,"I am always courteous, even to people who are ...",There have been occasions when I felt like sma...,I have never been irked when people expressed ...,There have been times when I was quite jealous...,I sometimes think when people have a misfortun...,I have never deliberately said something that ...,Random ID
1,"{""ImportId"":""startDate"",""timeZone"":""America/Lo...","{""ImportId"":""endDate"",""timeZone"":""America/Los_...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID15""}","{""ImportId"":""QID16""}","{""ImportId"":""QID17""}","{""ImportId"":""QID18""}","{""ImportId"":""QID19""}","{""ImportId"":""QID20""}","{""ImportId"":""QID21""}","{""ImportId"":""QID22""}","{""ImportId"":""QID23""}","{""ImportId"":""Random ID""}"
2,2021-07-07 13:22:52,2021-07-07 13:23:14,Survey Preview,NaN,100,21,True,2021-07-07 13:23:15,R_1gTIN3HfJ3nXBjd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8443833610
3,2021-07-07 15:37:48,2021-07-07 15:38:03,Survey Preview,NaN,100,14,True,2021-07-07 15:38:04,R_33kBZiT7hENv2Su,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2407293630
4,2021-07-08 17:05:50,2021-07-08 17:07:15,IP Address,65.183.131.208,100,85,True,2021-07-08 17:07:16,R_3O2JyxG1MVkaDyb,NaN,...,True,True,True,True,False,True,True,False,True,1034343201


In [39]:
# get names of columns
print(sd.columns)

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5',
       'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Random ID'],
      dtype='object')


In [40]:
# drop unnecessary unnecessary columns in sd
sd = sd.drop(['Progress','StartDate','EndDate', 'Status', 'IPAddress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
                  'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude',
                 'DistributionChannel', 'UserLanguage'], axis = 1)


In [41]:
# drop unnecessary rows
sd.dropna(subset = ["Random ID"], inplace = True)
sd = sd.iloc[2:]

In [42]:
# take only rows with non Nan values?
# df = df[df['EPS'].notna()]
sd = sd[sd['Q1'].notna()==True]

In [43]:
# drop duplicates
sd = sd.drop_duplicates(subset= 'Random ID', keep='first', inplace=False)

In [44]:
# reset index to "Random ID"
sd.set_index(["Random ID"], inplace = True, append = True, drop = True)
sd.reset_index(inplace = True)
sd.head(10)

,level_0,Random ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17
0,4,1034343201,Male,18 - 24,White or Caucasian,Community member,True,True,True,True,True,True,True,True,False,True,True,False,True
1,5,9025272566,Female,55 - 64,White or Caucasian,Community member,True,False,False,True,False,False,False,True,False,False,True,False,True
2,6,3489602219,Male,25 - 34,A race/ethnicity not listed here,Community member,False,True,True,True,False,True,False,True,False,True,False,False,True
3,7,7124352876,Male,18 - 24,Asian or Pacific Islander,Community member,False,True,True,True,False,True,True,False,True,False,True,False,False
4,8,7671412241,Female,25 - 34,White or Caucasian,Community member,False,True,True,True,True,True,True,False,False,True,True,True,False
5,9,6603259763,Female,18 - 24,White or Caucasian,Community member,True,True,True,True,True,True,False,True,True,False,True,False,False
6,10,2782470529,Male,25 - 34,White or Caucasian,Community member,True,False,True,True,True,True,False,True,True,False,False,False,True


In [45]:
# drop "level_0" column
sd = sd.drop(['level_0'], axis = 1)

### Recode True/False SD Answers

Recode True/False Answers to 1/0 depending on if something scores "yes" (i.e. 1) to socially desirable or "no" (i.e. 0) to socially desirable



Where the answer key is as follows:

Q5: True
Q6: False
Q7: False
Q8: False
Q9: False
Q10: False
Q11: False
Q12: True
Q13: False
Q14: True
Q15: False
Q16: False
Q17: True

In [46]:
# recode True/False Questions to 1 for correct (i.e. correct for high social desirability) answers and 0 for wrong answers (i.e. correct for high social desirability)

# Q5
sd.loc[sd.Q5 == 'True', 'Q5'] = 1
sd.loc[sd.Q5 == 'False', 'Q5'] = 0

# Q6
sd.loc[sd.Q6 == 'True', 'Q6'] = 0
sd.loc[sd.Q6 == 'False', 'Q6'] = 1

# Q7
sd.loc[sd.Q7 == 'True', 'Q7'] = 0
sd.loc[sd.Q7 == 'False', 'Q7'] = 1

# Q8
sd.loc[sd.Q8 == 'True', 'Q8'] = 0
sd.loc[sd.Q8 == 'False', 'Q8'] = 1

# Q9
sd.loc[sd.Q9 == 'True', 'Q9'] = 0
sd.loc[sd.Q9 == 'False', 'Q9'] = 1

# Q10
sd.loc[sd.Q10 == 'True', 'Q10'] = 0
sd.loc[sd.Q10 == 'False', 'Q10'] = 1

# Q11
sd.loc[sd.Q11 == 'True', 'Q11'] = 0
sd.loc[sd.Q11 == 'False', 'Q11'] = 1

# Q12
sd.loc[sd.Q12 == 'True', 'Q12'] = 1
sd.loc[sd.Q12 == 'False', 'Q12'] = 0

# Q13
sd.loc[sd.Q13 == 'True', 'Q13'] = 0
sd.loc[sd.Q13 == 'False', 'Q13'] = 1

# Q14
sd.loc[sd.Q14 == 'True', 'Q14'] = 1
sd.loc[sd.Q14 == 'False', 'Q14'] = 0

# Q15
sd.loc[sd.Q15 == 'True', 'Q15'] = 0
sd.loc[sd.Q15 == 'False', 'Q15'] = 1

# Q16
sd.loc[sd.Q16 == 'True', 'Q16'] = 0
sd.loc[sd.Q16 == 'False', 'Q16'] = 1

# Q17
sd.loc[sd.Q17 == 'True', 'Q17'] = 1
sd.loc[sd.Q17 == 'False', 'Q17'] = 0

In [47]:
# create social desirability score by adding columns Q5, Q6, Q7, Q8, Q9, Q10, Q11, Q12, Q13, Q14, Q15, Q16, and Q17, 
# divding by 13 questions, and subtracting this value from 1
sd['sd_score'] = 1-((sd['Q5'] + sd['Q6'] + sd['Q7'] + sd['Q8'] + sd['Q9'] + sd['Q10'] + 
                      sd['Q11'] + sd['Q12'] + sd['Q13'] + sd['Q14'] + sd['Q15'] + sd['Q16'] + sd['Q17'])/13)

In [48]:
# gain insight into sd_copy

# where a high sd_score indicates that your answers for attitudes and practices can be taken at more or less face value
# and where a low sd_score indicates that your answers for attitudes and practices cannot be taken more or less at face value 
sd.head()

,Random ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,sd_score
0,1034343201,Male,18 - 24,White or Caucasian,Community member,1,0,0,0,0,0,0,1,1,1,0,1,1,0.538462
1,9025272566,Female,55 - 64,White or Caucasian,Community member,1,1,1,0,1,1,1,1,1,0,0,1,1,0.230769
2,3489602219,Male,25 - 34,A race/ethnicity not listed here,Community member,0,0,0,0,1,0,1,1,1,1,1,1,1,0.384615
3,7124352876,Male,18 - 24,Asian or Pacific Islander,Community member,0,0,0,0,1,0,0,0,0,0,0,1,0,0.846154
4,7671412241,Female,25 - 34,White or Caucasian,Community member,0,0,0,0,0,0,0,0,1,1,0,0,0,0.846154


In [49]:
# drop unnecessary unnecessary columns in sd of Q5, Q6, Q7, Q8, Q9, Q10, Q11, Q12, Q13, Q14, Q15, Q16, and Q17
sd = sd.drop(['Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17'], axis = 1)

In [50]:
# rename Q1 (gender), Q2 (age), Q3 (race), and Q4 (profession) columns
sd.columns = ['RandomID', 'gender', 'age', 'race', 'profession', 'sd_score']

## Bring SD and KAP data together

In [51]:
# prepare "RandomID" column keys for merging
sd['RandomID'] = sd['RandomID'].astype(str)
kap12['RandomID'] = kap12['RandomID'].astype(str)

In [52]:
# merge sd data and kap12 data on "Random ID"
kap_data = pd.merge(kap12, sd, on='RandomID', how='outer')

In [53]:
# gain insight into kap_data
kap_data.head()

,RandomID,pre mean a_pc,pre mean a_er,pre mean p_pc,pre mean k_pc,post mean a_pc,post mean a_er,post mean p_pc,post mean k_pc,diff_a_pc,diff_a_er,diff_p_pc,diff_k_pc,gender,age,race,profession,sd_score
0,1034343201,2.333333,5.000000,4.333333,1.0,1.666667,5.000000,3.333333,1.000000,-0.666667,0.0,-1.000000,0.000000,Male,18 - 24,White or Caucasian,Community member,0.538462
1,3489602219,2.000000,3.333333,5.000000,1.0,2.000000,3.333333,4.000000,1.000000,0.000000,0.0,-1.000000,0.000000,Male,25 - 34,A race/ethnicity not listed here,Community member,0.384615
2,9025272566,3.333333,5.000000,2.666667,1.0,2.666667,5.000000,3.333333,0.666667,-0.666667,0.0,0.666667,-0.333333,Female,55 - 64,White or Caucasian,Community member,0.230769
3,7671412241,2.000000,4.666667,3.666667,1.0,2.333333,4.666667,3.333333,0.666667,0.333333,0.0,-0.333333,-0.333333,Female,25 - 34,White or Caucasian,Community member,0.846154
4,7124352876,2.000000,2.666667,2.333333,1.0,2.000000,2.666667,2.666667,1.000000,0.000000,0.0,0.333333,0.000000,Male,18 - 24,Asian or Pacific Islander,Community member,0.846154


In [31]:
# write kap_data to csv
kap_data.to_csv('data/interim/kap_sd_practice_clean.csv', encoding='utf-8', index=False)

## Bring SD and SGD data together

In [54]:
# gain insight into sgd
sgd.head()

,RandomID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,total mean sgd_effect,mean sgd_sf,mean sgd_u,mean sgd_tc
0,1034343201,5,5,5,4,5,4,5,5,5,3,5,4.636364,5.000000,4.50,4.50
1,3489602219,4,5,5,5,5,4,4,5,5,4,5,4.636364,4.666667,4.50,4.75
2,9025272566,5,5,5,5,5,5,4,5,5,4,5,4.818182,5.000000,4.75,4.75
3,6603259763,4,4,4,4,4,5,3,4,5,4,5,4.181818,4.000000,4.00,4.50
4,7671412241,4,2,5,4,2,5,4,5,2,4,3,3.636364,3.666667,3.75,3.50


In [55]:
# prepare "RandomID" column keys for merging
sd['RandomID'] = sd['RandomID'].astype(str)
sgd['RandomID'] = sgd['RandomID'].astype(str)

In [57]:
# merge sd data and sgd data on "Random ID"
sgd_data = pd.merge(sgd, sd, on='RandomID', how='outer')

In [58]:
# gain insight into sgd_data
sgd_data.head()

,RandomID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q11,total mean sgd_effect,mean sgd_sf,mean sgd_u,mean sgd_tc,gender,age,race,profession,sd_score
0,1034343201,5,5,5,4,5,4,5,5,5,...,5,4.636364,5.000000,4.50,4.50,Male,18 - 24,White or Caucasian,Community member,0.538462
1,3489602219,4,5,5,5,5,4,4,5,5,...,5,4.636364,4.666667,4.50,4.75,Male,25 - 34,A race/ethnicity not listed here,Community member,0.384615
2,9025272566,5,5,5,5,5,5,4,5,5,...,5,4.818182,5.000000,4.75,4.75,Female,55 - 64,White or Caucasian,Community member,0.230769
3,6603259763,4,4,4,4,4,5,3,4,5,...,5,4.181818,4.000000,4.00,4.50,Female,18 - 24,White or Caucasian,Community member,0.692308
4,7671412241,4,2,5,4,2,5,4,5,2,...,3,3.636364,3.666667,3.75,3.50,Female,25 - 34,White or Caucasian,Community member,0.846154


In [59]:
# write sgd_data to csv
sgd_data.to_csv('data/interim/sgd_sd_practice_clean.csv', encoding='utf-8', index=False)